In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import shape
from tqdm import tqdm

In [3]:
gdf = gpd.read_file("2021-04-08 - elec_df.gpkg")

In [4]:
collapsed_elec_gdf = gdf.groupby(by=['state', 'year', 'district']).mean().reset_index()

## Identifying districts which don't (or barely) change boundaries from one period to the next

Note that I could just 

In [5]:
district_changing_dict = {}

for i,x in gdf[['state', 'year', 'district', 'geometry']].iterrows():
    district_changing_dict[ x['state'] ] = {}
    
for i,x in gdf[['state', 'year', 'district', 'geometry']].iterrows():
    district_changing_dict[ x['state'] ][ x['district'] ] = {}
    
for i,x in gdf[['state', 'year', 'district', 'geometry']].iterrows():
    district_changing_dict[ x['state'] ][ x['district'] ][ x['year'] ] = x['geometry']

In [6]:
def percent_overlap(area1, area2):
    return (area1.intersection(area2).area) / (((area2.area) + (area1.area))/2)

In [18]:
# # thresholds = [.8]
# thresholds = [.7, .75, .8, .85, .9, .95]
# dist_didnt_change_dict = {thresh:{} for thresh in thresholds}

# # CURRENT VERSION - MOST PRECISE

# for threshold in dist_didnt_change_dict:

#     for state in tqdm(district_changing_dict):
#         print(state, threshold, '!!!!!!!!!!!!')
#         dist_didnt_change_dict[threshold][state] = {}    
        
#         for district in district_changing_dict[state]:
#             dist_didnt_change_dict[threshold][state][district] = {2000: 1}
#             print(f'Now attempting {district}:')
#             print(dist_didnt_change_dict[threshold][state])
#             for year in [2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]:
                
#                 try:
#                     if percent_overlap(district_changing_dict[state][district][year], district_changing_dict[state][district][year + 2]) >= threshold:
#                         dist_didnt_change_dict[threshold][state][district][year + 2] = 1
#                     else:
#                         dist_didnt_change_dict[threshold][state][district][year + 2] = 0

#                 except Exception as e:
#                     print(e)
#                     print(state, district, year)
#                     dist_didnt_change_dict[threshold][state][district][year + 2] = 0
#             print(f'Just completed {district}:')
#             print(dist_didnt_change_dict[threshold][state])

In [ ]:
import json
json.dump( dist_didnt_change_dict, open( "dist_didnt_change_dict.json", 'w' ) )

In [ ]:
year_to_year_overlap_dict = {}

# CURRENT VERSION - MOST PRECISE


for state in tqdm(district_changing_dict):
    print(state, threshold, '!!!!!!!!!!!!')
    year_to_year_overlap_dict[state] = {}    

    for district in district_changing_dict[state]:
        year_to_year_overlap_dict[state][district] = {2000: 1}
        print(f'Now attempting {district}:')
        print(year_to_year_overlap_dict[state])
        for year in [2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018]:

            try:
                year_to_year_overlap_dict[state][district][year + 2] = percent_overlap(district_changing_dict[state][district][year], district_changing_dict[state][district][year + 2])

            except Exception as e:
                print(e)
                print(state, district, year)
                year_to_year_overlap_dict[state][district][year + 2] = 0
        print(f'Just completed {district}:')
        print(year_to_year_overlap_dict[state])

In [53]:
import json
json.dump( year_to_year_overlap_dict[.8], open( "year_to_year_overlap_dict.json", 'w' ) )


## Prepare to create a PD df from this dictionary

In [63]:
# year_to_year_overlap_dict = year_to_year_overlap_dict[.7]

# pre_df_dict = {x:[] for x in [2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018, 2020]}
# pre_df_dict['state'] = []
# pre_df_dict['district'] = []

# for state in year_to_year_overlap_dict:
#     for district in year_to_year_overlap_dict[state]:
#         pre_df_dict['state'].append(state)
#         pre_df_dict['district'].append(district)

#         for year in year_to_year_overlap_dict[state][district]:
#             pre_df_dict[year].append(year_to_year_overlap_dict[state][district][year])


# boundary_change_pct_df = pd.DataFrame.from_dict(pre_df_dict)

# boundary_change_pct_df.drop([2016, 2018, 2020], axis=1, inplace=True)

In [83]:
pre_df_dict_2 = {'year':[], 'state': [], 'district': [], 'area_similarity':[]}

for state in year_to_year_overlap_dict:
    
    for district in year_to_year_overlap_dict[state]:
        
        for year in year_to_year_overlap_dict[state][district]:
            
            pre_df_dict_2['year'].append(year)
            pre_df_dict_2['state'].append(state)
            pre_df_dict_2['district'].append(district)
            
            pre_df_dict_2['area_similarity'].append(year_to_year_overlap_dict[state][district][year])
        

In [86]:
boundary_change_pct_df = pd.DataFrame.from_dict(pre_df_dict_2)

boundary_change_pct_df['area_difference'] = 1-boundary_change_pct_df['area_similarity']

In [89]:
boundary_change_pct_df.to_csv(r"Year-to-Year Boundary Changes.csv")

In [90]:
gdf = gdf.merge(boundary_change_pct_df, on=['year','state','district'], how='left')

In [91]:
gdf.to_file("2021-04-08 - elec_df.gpkg", driver="GPKG")

In [92]:
df = pd.DataFrame(gdf.drop('geometry', axis=1))

In [94]:
df.to_csv("elec_df - 4-13-2021 (with boundary changes).csv")